In [ ]:
from seismic import SeismicIndex
import numpy as np
import json

Build the index

In [ ]:

json_input_file = ""

index = SeismicIndex.build(json_input_file)

In [ ]:
print("Number of documents: ", index.len)
print("Avg number of non-zero components: ", index.nnz / index.len)
print("Dimensionality of the vectors: ", index.dim)

index.print_space_usage_byte()

Load queries

In [ ]:
queries_path = ""

queries = []
with open(queries_path, 'r') as f:
    for line in f:
        queries.append(json.loads(line))

MAX_TOKEN_LEN = 30
string_type  = f'U{MAX_TOKEN_LEN}'

queries_ids = np.ascontiguousarray(np.array([q['id'] for q in queries], dtype=string_type))

query_components = []
query_values = []

for query in queries:
    vector = query['vector']
    try:
        query_components.append(np.array(list(vector.keys()), dtype=string_type))
        query_values.append(np.array(list(vector.values()), dtype=np.float32))
    except:
        continue

In [ ]:
index.search("id", query_components[0], query_values[0], k=10, query_cut=5, heap_factor=0.5, n_knn=0, sorted=False)

Perform the search

In [ ]:
results = index.batch_search(
    queries_ids=queries_ids,
    query_components=query_components,
    query_values=query_values,
    k=10,
    query_cut=20,
    heap_factor=0.7,
    sorted=True,
    n_knn=0,
)

Evaluation

In [ ]:
!pip install ir_datasets ir_measures
import ir_datasets
import ir_measures
from ir_measures import RR

In [ ]:
ir_results = [ir_measures.ScoredDoc(query_id, doc_id, score) for r in results for (query_id, score, doc_id) in r]
qrels = ir_datasets.load('msmarco-passage/dev/small').qrels

ir_measures.calc_aggregate([RR@10], qrels, ir_results)